In [584]:
import pandas as pd
import fnmatch
import os

In [585]:
# Load Data

In [586]:
path = '/Users/matthiashugli/Virtualenvs/youth-base/youth-base/training-analysis/data'

data = [os.path.join(dirpath, f) for dirpath, dirnames, files in os.walk(path) for f in fnmatch.filter(files, '*.xlsx')]

df = pd.DataFrame()
for filename in data:
    source = pd.read_excel(filename, sheet_name='s1_core_trm_player_unit_skills', index_col=None, header=1)
    source = source.iloc[2:]
    source = source.rename(columns=lambda c: c.replace('.1', '.count') if fnmatch.fnmatch(c, '*.1') else c + ".duration")
    source = source.melt(source.columns[0:3])
    source[['skills', 'entity']] = source.variable.apply(lambda x: pd.Series(str(x).split('.')))
    source = source.drop(columns=['Category Name.duration', 'variable'])
    source = source.rename(columns={'Unnamed: 0.duration': 'team', 'Unnamed: 1.duration': 'keeper'})
    source = source.pivot(index=['team', 'keeper', 'skills'], columns='entity', values='value').reset_index()
    source[['team', 'club']] = source.team.str.split(' BSC', 1, expand=True)
    source[['team', 'club']] = source.team.str.split(' YB', 1, expand=True)
    source['filename'] = filename
    source[['file', 'type']] = source.filename.apply(lambda x: pd.Series(str(x).split('.')))
    source['yearmonth'] = source.file.str[-6:]
    source['date'] = pd.to_datetime(source.yearmonth, format='%Y%m', errors='coerce').dropna()
    source['year'] = pd.DatetimeIndex(source['date']).year
    source['month'] = source.date.dt.month_name()
    source = source.drop(columns=['filename', 'file', 'type', 'club'])
    source = source.dropna(subset=['count', 'duration'])
    df = df.append(source)

df.to_csv('trainings.csv')
